In [12]:
%run /home/sebi/code/analysis/analysis-scripts/induction_analysis/group_scripts/init_all.py

[(0, 'SCE'), (1, 'Sparse Targeted'), (2, 'Dense'), (3, 'Dense + Dreadd'), (4, 'Sparse LED')]


In [27]:
import Analysis.induction_analysis as ia

%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style='ticks')

import numpy as np
from scipy.stats import sem

import pandas as pd

In [110]:
def formatPlot(ax, offset=10):
    for axis in ['left', 'right', 'bottom', 'top']:
        ax.spines[axis].set_linewidth(2)
    ax.tick_params(labelsize=14, width=2, length=4)
    sns.despine(ax=ax, offset=offset)
    ax.set_xlabel('')
    ax.set_ylabel('')

In [206]:
grp_means = []
grp_sems = []
grp_psths = []
Ts = []

for grp, stim_filter in zip(grps, stim_filters):

    psths = []
    
    expt_Ts = []
        
    for expt in grp:
        
        T, roi_psths = ia.psth(expt, roi_filter=stim_filter, pre=4, post=8)
        expt_Ts.extend([T] * roi_psths.shape[1])
        psths.extend(np.nanmean(roi_psths, axis=0))
        # By Cell For Now
        
    # Interpolate to make all experiments of same type same length
    anchor_idx = np.argmin([len(x) for x in expt_Ts])
    psths_interp = np.vstack([np.interp(expt_Ts[anchor_idx], T, rpsth) for T, rpsth in zip(expt_Ts, psths)])
    
    Ts.append(expt_Ts[anchor_idx])
    psths = np.vstack(psths_interp)

    psth_mean = np.nanmean(psths, axis=0)
    psth_sem = sem(psths, axis=0)
    
    grp_psths.append(psths)
    grp_means.append(psth_mean)
    grp_sems.append(psth_sem)

In [221]:
fig = plt.figure()
ax = fig.add_subplot(111)

for pmean, psem, T, label, color in zip(grp_means, grp_sems, Ts, labels, colors):
    
    ax.fill_between(T, pmean+psem, pmean-psem, color=color, alpha=0.3)
    ax.plot(T, pmean, color=color, label=label)
    
formatPlot(ax)
ax.set_ylabel('Normalized Fluorescence', fontsize=18)
ax.set_xlabel('Time From Stim Onset', fontsize=18)

# ax.set_xlim([-2, 6])

ax.legend()

fig.savefig('/home/sebi/plots/ind/figs1/mean_psth_by_condition.svg', bbox_inches='tight')

FigureCanvasNbAgg()